In [1]:
import os
os.chdir(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\ML Lab Spring 25\timeseires')


In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [3]:
class PC:
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        lstm=Bidirectional(LSTM(48,return_sequences=True))(inputs)
        lstm=LSTM(48)(lstm)
        lstm=Activation('tanh')(lstm)
        lstm=Dense(24)(lstm)
        # create the model
        model = Model(inputs, lstm, name="PC")
        
        # return the constructed network architecture
        return model

In [4]:
model1 =  PC.build(time_steps=24, num_features=21, reg=0.005)

In [5]:
model1.summary()

Model: "PC"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 21)]          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 24, 96)            26880     
_________________________________________________________________
lstm_1 (LSTM)                (None, 48)                27840     
_________________________________________________________________
activation (Activation)      (None, 48)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                1176      
Total params: 55,896
Trainable params: 55,896
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoints = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\\E3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.json"])

In [7]:
model = None
start_epoch = 0
time_steps=24
num_features=21

In [8]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [9]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [10]:
import os
path_dataset =r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\FYP DETAILS\dada sets\AEP\AEP'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [11]:
time_steps=24
num_features=21

In [12]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=24)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=24)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=24)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 7.215815782546997 sec


In [13]:
epochs = 2
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/2
2652/2652 [==============================] - ETA: 0s - loss: 0.0460 - mae: 0.0460 - mape: 2443.7163
Epoch 00001: val_loss improved from inf to 0.04022, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E3-cp-0001-loss0.04.h5
2652/2652 [==============================] - 427s 161ms/step - loss: 0.0460 - mae: 0.0460 - mape: 2443.7163 - val_loss: 0.0402 - val_mae: 0.0402 - val_mape: 17.3933
Epoch 2/2
2652/2652 [==============================] - ETA: 0s - loss: 0.0360 - mae: 0.0360 - mape: 2078.3560
Epoch 00002: val_loss improved from 0.04022 to 0.03538, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E3-cp-0002-loss0.04.h5
2652/2652 [==============================] - 393s 148ms/step - loss: 0.0360 - mae: 0.0360 - mape: 2078.3560 - val_loss: 0.0354 - val_mae: 0.0354 - val_mape: 14.6993


In [14]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E3-cp-0001-loss0.04.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 646.73
Median Absolute Error (MedAE): 512.43
Mean Squared Error (MSE): 710337.74
Root Mean Squared Error (RMSE): 842.82
Mean Absolute Percentage Error (MAPE): 4.38 %
Median Absolute Percentage Error (MDAPE): 3.57 %


y_test_unscaled.shape=  (12082, 24)
y_pred.shape=  (12082, 24)


In [15]:
checkpoints = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E3-cp-0001-loss0.04.h5'
start_epoch= 17

In [16]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E3-cp-0001-loss0.04.h5...


[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [17]:
epochs = 1
verbose = 1 #0
batch_size = 128
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

663/663 [==============================] - ETA: 0s - loss: 0.0349 - mean_absolute_error: 0.0349 - mean_absolute_percentage_error: 2165.7622
Epoch 00001: val_loss improved from inf to 0.03754, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E2-cp-0001-loss0.04.h5
663/663 [==============================] - 199s 301ms/step - loss: 0.0349 - mean_absolute_error: 0.0349 - mean_absolute_percentage_error: 2165.7622 - val_loss: 0.0375 - val_mean_absolute_error: 0.0375 - val_mean_absolute_percentage_error: 16.1430


In [21]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab2\E2-cp-0001-loss0.04.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 598.86
Median Absolute Error (MedAE): 462.79
Mean Squared Error (MSE): 622851.43
Root Mean Squared Error (RMSE): 789.21
Mean Absolute Percentage Error (MAPE): 4.05 %
Median Absolute Percentage Error (MDAPE): 3.23 %


y_test_unscaled.shape=  (12082, 24)
y_pred.shape=  (12082, 24)
